In [1]:
# # Importing relevant packages and establishing the Exadata connection

# import sqlalchemy
# import pandas as pd
# import numpy as np
# import getpass

# password=getpass.getpass("Enter your password:")
# engine = sqlalchemy.create_engine("oracle+cx_oracle://ashwinik[an_rt_ws18]:%s@exa_gbtukprdd_an" %(password))


Enter your password:········


/usr/lib64/python2.7/site-packages/sqlalchemy/dialects/oracle/cx_oracle.py:753: SAWarning: cx_Oracle is compiled under Python 2.xx using the WITH_UNICODE flag.  Consider recompiling cx_Oracle without this flag, which is in no way necessary for full support of Unicode. Otherwise, all string-holding bind parameters must be explicitly typed using SQLAlchemy's String type or one of its subtypes,or otherwise be passed as Python unicode.  Plain Python strings passed as bind parameters will be silently corrupted by cx_Oracle.
  "cx_Oracle is compiled under Python 2.xx using the "


In [2]:


# In[2]:


## Using xdatadump to get transactions from exadata table

# import subprocess
# import os
# from urllib import quote_plus as urlquote

# # DROP TABLE AK_EXP_TRANS;
# # CREATE TABLE AK_EXP_TRANS PARALLEL COMPRESS FOR QUERY HIGH AS 
# # SELECT TRANSACTION_FID,CONVERTED_PROD_GROUP_ID AS TPNB, 1 AS VALUE
# # FROM SMP10_TRAN_ITEM_FCT A 
# # INNER JOIN PROD_DIM_C B ON A.PROD_ID = B.PROD_ID
# # INNER JOIN DATE_DIM D ON A.DATE_ID = D.DATE_ID
# # INNER JOIN STORE_DIM_C C ON A.STORE_ID = C.STORE_ID 
# # WHERE PROD_MERCH_L20_CODE IN ('S1F','B1B','F1A','D2A','D2P','U2N','S3A','G3C','G1C','G1F','S3K','P3E','G1T')
# # AND FIS_WEEK_ID BETWEEN 201649 AND 201652 AND FORMAT_CODE = 'X'; 

# proxy = "ashwinik[an_rt_ws18]/%s@exa_gbtukprdd_an" %(password)

# cmd_str =   "xdatadump userid='" + str(proxy)                 + "' sqlstmt='SELECT  * FROM  AK_final_test "                 +  "' delimiter=^ header=y > exp_trans_13wk.csv"        

# print cmd_str
# subprocess.call(cmd_str, cwd='./', shell=True)



xdatadump userid='ashwinik[an_rt_ws18]/Welcome16@exa_gbtukprdd_an' sqlstmt='SELECT  * FROM  AK_final_test ' delimiter=^ header=y > exp_trans_13wk.csv


0

In [3]:

# # In[3]:


# # Importing the transaction file to be used to create the adjacency matrix

# # tp = pd.read_csv('exp_trans_13wk.csv',header=0, delimiter="^",chunksize = 10000, iterator=False)
# # df = pd.concat(tp, ignore_index=True)

# df1 = pd.read_csv('exp_trans_13wk.csv',header=0, delimiter="^").groupby(['TRANSACTION_FID','TPNB'])['VALUE'].max().unstack()



# # In[4]:


# df1.shape



(1742404, 3687)

In [ ]:

# # In[5]:


# # print df1.head()
# # print df1.describe()

# # df1
# # df1.fillna(0,inplace = True)
# X = np.array(df1)
# X[~np.isfinite(X)]= 0



In [1]:

# In[8]:


import time
from sklearn.decomposition import NMF, LatentDirichletAllocation
from scipy import sparse

from scipy import sparse, io

X_sparse = io.mmread("test.mtx")

# X = df1.as_matrix()
# X_sparse = sparse.csr_matrix(X)

print "Fitting the NMF model for 1000 components"  

t0 = time.time()

nmf = NMF(n_components=1000, random_state=1,
          alpha= 0,verbose = 1, l1_ratio= 0,init = 'nndsvd')
nmf.fit(X_sparse)

print("done in %0.3fs." % (time.time() - t0))


# In[ ]:

Fitting the NMF model for 1000 components
violation: 1.0
violation: 0.0107537182211
violation: 0.000613995865981
violation: 3.49011101596e-05
Converged at iteration 4
done in 10.970s.


In [2]:

# In[9]:


import pickle


with open('nmf.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(nmf, f)

